# WFM Labor Intelligence Suite — Cost Center Schedule vs Actual (Daily)

**Goal:** Identify where labor is drifting from plan by **cost center** and **day**, and quantify:
- **Scheduled vs Worked hours**
- **Variance (hrs)** and **Variance (%)**
- **Top drivers** of labor leakage

**Audience:** Workforce leadership, operations leaders, finance partners  
**Data:** Synthetic (safe) — generated from `synthetic_data_engine/`  

In [8]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.defaults.default_format = "png"
pio.templates.default = "plotly_white"

cwd = Path(os.getcwd()).resolve()
repo = None
p = cwd
for _ in range(10):
    if (p / "README.md").exists() and (p / "data").exists():
        repo = p
        break
    p = p.parent

if repo is None:
    raise FileNotFoundError("Could not detect repo root (expected README.md + data/).")

DATA_DIR = repo / "data" / "synthetic_raw"
VIZ_DIR = repo / "viz"
VIZ_DIR.mkdir(exist_ok=True)

print("Repo:", repo)
print("Data:", DATA_DIR)
print("Viz :", VIZ_DIR)


Repo: C:\Temp\wfm-labor-intelligence-suite
Data: C:\Temp\wfm-labor-intelligence-suite\data\synthetic_raw
Viz : C:\Temp\wfm-labor-intelligence-suite\viz


In [2]:
employees = pd.read_csv(DATA_DIR / "employees.csv")
schedules  = pd.read_csv(DATA_DIR / "schedules.csv")
timecards  = pd.read_csv(DATA_DIR / "timecards.csv")

schedules["schedule_date"] = pd.to_datetime(schedules["schedule_date"])
timecards["work_date"] = pd.to_datetime(timecards["work_date"])

summary = pd.DataFrame([
    {"Dataset": "employees", "Rows": employees.shape[0], "Columns": employees.shape[1]},
    {"Dataset": "schedules",  "Rows": schedules.shape[0],  "Columns": schedules.shape[1]},
    {"Dataset": "timecards",  "Rows": timecards.shape[0],  "Columns": timecards.shape[1]},
])

(summary
 .style
 .hide(axis="index")
 .set_table_styles([
     {"selector": "th", "props": [("text-align", "left"), ("font-size", "14px")]},
     {"selector": "td", "props": [("text-align", "left"), ("font-size", "14px"), ("font-weight", "600")]}
 ])
 .format({"Rows": "{:,}", "Columns": "{:,}"})
)

Dataset,Rows,Columns
employees,350,11
schedules,"14,287",10
timecards,"13,849",11


## Method

We calculate daily labor metrics by cost center:

- **Scheduled Hours**: sum of scheduled assigned shift hours (excluding open shifts)
- **Worked Hours**: sum of clocked/paid hours from timecards
- **Variance (hrs)** = Worked − Scheduled  
- **Variance (%)** = Variance ÷ Scheduled

We then rank cost centers by total worked hours and analyze the **Top 25 cost centers**.

In [3]:
sched_daily_all = (
    schedules[~schedules["is_open_shift"].astype(bool)]
    .groupby(["org_path","schedule_date"], as_index=False)
    .agg(scheduled_hours=("scheduled_hours","sum"))
)

work_daily_all = (
    timecards
    .groupby(["org_path","work_date"], as_index=False)
    .agg(worked_hours=("worked_hours","sum"))
    .rename(columns={"work_date":"schedule_date"})
)

daily_all = sched_daily_all.merge(work_daily_all, on=["org_path","schedule_date"], how="outer").fillna(0)
daily_all["variance_hours"] = (daily_all["worked_hours"] - daily_all["scheduled_hours"]).round(2)
daily_all["variance_pct"] = np.where(
    daily_all["scheduled_hours"] > 0,
    (daily_all["variance_hours"] / daily_all["scheduled_hours"]) * 100,
    0.0
).round(2)

daily_all.head()

,org_path,schedule_date,scheduled_hours,worked_hours,variance_hours,variance_pct
0,HHS/TX/FacilityA/EVS,2025-10-08,192.0,193.22,1.22,0.64
1,HHS/TX/FacilityA/EVS,2025-10-09,152.0,149.50,-2.50,-1.64
2,HHS/TX/FacilityA/EVS,2025-10-10,144.0,141.08,-2.92,-2.03
3,HHS/TX/FacilityA/EVS,2025-10-11,160.0,156.76,-3.24,-2.02
4,HHS/TX/FacilityA/EVS,2025-10-12,152.0,158.31,6.31,4.15


## Executive Summary (Top 25 Cost Centers)

These KPIs summarize variance across the highest-volume cost centers.

In [10]:
top_n = 25
orgs_to_show = (
    daily_all.groupby("org_path")["worked_hours"].sum()
    .sort_values(ascending=False)
    .head(top_n)
    .index
    .tolist()
)

scope = daily_all[daily_all["org_path"].isin(orgs_to_show)].copy()

summary = {
    "Cost Centers": len(orgs_to_show),
    "Days": int(scope["schedule_date"].nunique()),
    "Total Scheduled Hours": float(scope["scheduled_hours"].sum().round(2)),
    "Total Worked Hours": float(scope["worked_hours"].sum().round(2)),
    "Net Variance (hrs)": float(scope["variance_hours"].sum().round(2)),
    "Avg Daily Variance (hrs)": float(scope.groupby("schedule_date")["variance_hours"].sum().mean().round(2)),
}

pd.DataFrame([summary]).style.hide(axis="index").set_properties(**{"font-size":"14px"})

Cost Centers,Days,Total Scheduled Hours,Total Worked Hours,Net Variance (hrs),Avg Daily Variance (hrs)
5,90,145468.000000,145597.210000,129.210000,1.440000


## Scheduled vs Worked Trend (Top 25 Cost Centers)

Dashed = Scheduled, Solid = Worked.

In [9]:
melted = scope.melt(
    id_vars=["org_path","schedule_date"],
    value_vars=["scheduled_hours","worked_hours"],
    var_name="Metric",
    value_name="Hours"
)
melted["Metric"] = melted["Metric"].replace({
    "scheduled_hours": "Scheduled",
    "worked_hours": "Worked"
})

fig = px.line(
    melted,
    x="schedule_date",
    y="Hours",
    color="org_path",
    line_dash="Metric",
    title="Scheduled vs Worked Hours — Top 25 Cost Centers"
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Hours",
    hovermode="x unified",
    legend_title_text="Cost Center",
)

trend_path = VIZ_DIR / "multi_cost_center_trend.png"
fig.write_image(trend_path, scale=2)

fig.show()
trend_path

WindowsPath('C:/Temp/wfm-labor-intelligence-suite/viz/multi_cost_center_trend.png')

## Heatmap: Daily Variance (hrs) by Cost Center

This view surfaces recurring drift patterns and outlier days instantly.

In [11]:
pivot = (
    scope.pivot_table(
        index="schedule_date",
        columns="org_path",
        values="variance_hours",
        aggfunc="sum",
        fill_value=0
    )
    .sort_index()
)

order = pivot.abs().sum().sort_values(ascending=False).index
pivot = pivot[order]

heat = px.imshow(
    pivot.T,
    aspect="auto",
    title="Daily Variance (hrs) Heatmap — Top 25 Cost Centers",
    labels={"x":"Date","y":"Cost Center","color":"Variance (hrs)"}
)

heat.update_layout(height=750)

heat_path = VIZ_DIR / "variance_heatmap.png"
heat.write_image(heat_path, scale=2)

heat.show()
heat_path

WindowsPath('C:/Temp/wfm-labor-intelligence-suite/viz/variance_heatmap.png')

## Action Table: Top 50 Variance Days (Absolute)

This table is designed for operational review: identify the biggest drift events.

In [7]:
worst_days = (
    scope.assign(abs_variance=lambda x: x["variance_hours"].abs())
    .sort_values("abs_variance", ascending=False)
    .head(50)
    .rename(columns={
        "org_path": "Cost Center",
        "schedule_date": "Date",
        "scheduled_hours": "Scheduled (hrs)",
        "worked_hours": "Worked (hrs)",
        "variance_hours": "Variance (hrs)",
        "variance_pct": "Variance (%)",
    })
)

(worst_days[["Date","Cost Center","Scheduled (hrs)","Worked (hrs)","Variance (hrs)","Variance (%)"]]
 .style.hide(axis="index")
 .format({
     "Date": lambda d: pd.to_datetime(d).strftime("%Y-%m-%d"),
     "Scheduled (hrs)": "{:,.2f}",
     "Worked (hrs)": "{:,.2f}",
     "Variance (hrs)": "{:,.2f}",
     "Variance (%)": "{:,.2f}%"
 })
)

Date,Cost Center,Scheduled (hrs),Worked (hrs),Variance (hrs),Variance (%)
2025-11-08,HHS/TX/FacilityA/Nursing,564.00,585.16,21.16,3.75%
2025-10-24,HHS/TX/FacilityA/HR,72.00,91.34,19.34,26.86%
2025-11-18,HHS/TX/FacilityB/EVS,144.00,160.85,16.85,11.70%
2025-11-21,HHS/TX/FacilityB/EVS,176.00,192.48,16.48,9.36%
2025-10-10,HHS/TX/FacilityB/EVS,168.00,183.92,15.92,9.48%
2025-11-04,HHS/TX/FacilityB/EVS,160.00,175.47,15.47,9.67%
2025-10-10,HHS/TX/FacilityA/HR,80.00,95.30,15.30,19.12%
2025-11-28,HHS/TX/FacilityA/HR,112.00,126.81,14.81,13.22%
2025-10-30,HHS/TX/FacilityA/HR,88.00,102.78,14.78,16.80%
2025-11-23,HHS/TX/FacilityA/HR,48.00,62.53,14.53,30.27%


## Executive Takeaways

- Identify cost centers with **consistent overage** (recurring variance, not just one-off outliers).
- Pair heatmap outliers with manager review: “Was this a staffing shortage, late call-out, or schedule edit?”
- Next step: break variance into drivers (OT, unscheduled work, call pay, etc.) and flag high-risk patterns.